# 라이브러리

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

# 셀레니윰으로 링커리어 자소서 접근

In [2]:
# 크롬 브라우저 실행
browser = webdriver.Chrome()
# 창 키우기
browser.maximize_window()

# 접근할 url (링커리어)
url = 'https://linkareer.com/'

page_cnt = 1

# 자소서 정보 데이터 프레임 만들기
df = pd.DataFrame(columns=['기업명', '모집전형', '모집시기', '대학교', '전공', '학점', '기타스펙', '자소서'])

# 자소서 page 순서대로 가져오기
for n in range(1,5):
    start_url = url + f'cover-letter/search?page={n}&tab=all'
    browser.get(start_url)
    time.sleep(1)

    # 현재 html 웹 페이스 문서 소스 가져오기
    recent_web = bs(browser.page_source, 'lxml')

    # 현 페이지의 자소서 링크 가져오기 (원래는 div.MuiBox-root.jss167.jss168 이부분에서 jss숫자 이게 3개 인데, 맨 앞에만 달라지고, 뒤 2개가 공통분모여서 공통된 요소만 남겨둠)
    atag_list = recent_web.select('#__next > div.MuiBox-root.jss3 > div > div.MuiContainer-root.jss12.jss1.MuiContainer-disableGutters > div.MuiBox-root.jss154.jss148 > div > div.MuiBox-root.jss165.jss152 > div.MuiContainer-root.jss150.MuiContainer-disableGutters.MuiContainer-maxWidthLg > div.MuiBox-root.jss167.jss168 > div.jss174 > a')

    for atag in atag_list:

        # 자소서 링크
        new_url = url + atag.get('href')[1:] + '&simple=true'

        # 현 페이지에서 각 자소서 링크별로 접속하기
        browser.get(new_url)
        time.sleep(1)

        # 자소서 내용이 답긴 웹 페이지
        contents_soup = bs(browser.page_source, 'lxml')

        # 기업명 / 모집전형 / 모집시기
        intro1 = contents_soup.select('#__next > div.jss6.jss2 > div.MuiContainer-root.jss14.jss3.MuiContainer-disableGutters > div > div > div.MuiContainer-root.jss98.jss99.MuiContainer-maxWidthLg > div > div.MuiBox-root.jss160.jss152 > div > div > div.MuiBox-root.jss168')
        # 대학교 / 전공 / 학점 / 기타스펙
        intro2 = contents_soup.select('#__next > div.jss6.jss2 > div.MuiContainer-root.jss14.jss3.MuiContainer-disableGutters > div > div > div.MuiContainer-root.jss98.jss99.MuiContainer-maxWidthLg > div > div.MuiBox-root.jss160.jss152 > div > div > div.MuiBox-root.jss169')
        # 자기소개서
        texts = contents_soup.select('#coverLetterContent > main')

        # 양식이 안 맞는 경우가 있기에 조정

        # 기업명이 'sk텔레콤 / 5G' 이런식으로 구성돼 있는 경우가 있기에
        new_info1 = intro1[0].get_text().split(' / ')
        if len(new_info1) != 3 :
            new_info1[0] = ' '.join(new_info1[:2])
            del new_info1[1]

        # 대학교 / 전공 / 학점 까지는 공통적인데 그 뒤로 다양한 형태여서 통일
        new_info2 = intro2[0].get_text().split(' / ')
        new_info2[3] = ' '.join(new_info2[3:])
        del new_info2[4:]

        # 전체 한 리스트로 합체
        info = new_info1 + new_info2 + [texts[0].get_text()]

        # 우리의 자소서 데이터 프레임에 추가 하기
        df = df.append(pd.Series(info, index=df.columns), ignore_index=True)

browser.quit()
    

# 웹스크래핑한 데이터프레임 

In [3]:
df

,기업명,모집전형,모집시기,대학교,전공,학점,기타스펙,자소서
0,북부산 새마을금고,은행원(시중은행) 인턴,2021 상반기,인제대,문과,학점 3.3,"컴퓨터활용능력 : 1급, 모스마스터","문장 스크랩복사공유지원동기\n\n저는 대학교 1학년 때, 아울렛에 있는 한 의류매장..."
1,한국수력원자력,사무직 인턴,2021 상반기,동국대학교,경영학과,학점 3.5,"토익 : 800 리디북스 재무팀 한국사검정시험 : 고급, 컴퓨터활용능력 : 1급, ...",문장 스크랩복사공유1. 체험형 청년인턴 지원 동기 및 입사 후 포부에 대해 작성해 ...
2,동원그룹,생산관리,2021 상반기,아주대학교,산업공학과,학점 3.30,"오픽 : IM2 한국사검정시험 : 고급, 품질경영기사",문장 스크랩복사공유* 1. 나의 '인생 가치관'에 대해 성장과정 또는 경험을 바탕으...
3,CJ대한통운,CL(계약물류),2021 상반기,지거국,경영,학점 3.54,"토익: , 토익스피킹: 7, 오픽: , 기타: 사회생활 경험: 인턴 한국사검정시험...",문장 스크랩복사공유함께라면 물류 전문가로 성장할 수 있는 꿈의 파트너\n물류의 본질...
4,현대엔지니어링,자산관리 자산설비,2021 상반기,한양대 ERICA,기계공학과,학점 3.78/4.5,"토익: 855, 토익스피킹: 130/레벨6 사회생활 경험: 한국건설기술연구원 2개월...","문장 스크랩복사공유본인이 회사를 선택할때의 기준은 무엇이며, 왜 현대엔지니어링이 그..."
...,...,...,...,...,...,...,...,...
75,ASML,CS 엔지니어,2021 상반기,고려대학교(세종),전자및정보공학과,학점 3.47/4.5,"토익: , 토익스피킹: , 오픽: IM2, 기타: 사회생활 경험: 유라코퍼레이션 ...",문장 스크랩복사공유1. 수상경험 / 프로젝트 / 공모전 / 대회 / 연구 및 기타 ...
76,코오롱인더스트리,영업,2021 상반기,연세대,불어불문/동아시아학,학점 3.61,"토익: 950, 토익스피킹: , 오픽: Ih, 기타: Jlpt N2, delf B2...",문장 스크랩복사공유1. 우리회사와 희망직무를 선택하게 된 동기를 기술해 주시기 바랍...
77,초록우산,"홍보,모금",2021 상반기,울산대학교,중어중문학과,학점 3.29,"토익: 855, 토익스피킹: , 오픽: IH, 기타: HSK6급 사회생활 경험: 수...","문장 스크랩복사공유1. 지원한 직무와 관련한 본인의 비전, 목표 등을 기술해주십시오..."
78,서울신문사,미디어경영,2021 상반기,한양대에리카,경영학부/산업경영공학과,학점 3.65/4.5,"토익: 830, 토익스피킹: , 오픽: IH, 기타: 사회생활 경험: 인턴1회, ...",문장 스크랩복사공유1. 살면서 본인 인생의 전환점 또는 기회가 됐던 사건이나 인물 ...
